# Read and Copy the dimensions


In [3]:
import numpy as np
import netCDF4
import os

NEWFILE_PATH = 'newfiletest.nc'

WRFOUT_FILE_PATH = 'wrfout_d01_2017-09-09_12:00:00'
#Open original for reading and new diagnostic for writing
orig_dataset = netCDF4.Dataset(NEWFILE_PATH, 'r')

orig_dataset.close